# SrFit

The diffpy.srfit package provides a top-level framework for setting up multi-component fits.  The SrFit recipes tune parameters in the optimized complexes and link them to refinable variables, which can then be tied to functional constraints or restraints or fixed at a constant values.

This notebook will demonstrate these features by setting up a SrFit recipe to manage a two-component fit.  To start with we'll generate a single data stream with some noisy linear data and pass it to a *Profile* object, a SrFit class for representing observed data.

In [1]:
from __future__ import print_function
import numpy as np
A = 3
B = 5
noise_level = 3.0
xobs0 = np.linspace(0, 10.0, 21)
yobs0 = A * xobs0 + B + noise_level * np.random.randn(xobs0.size)
dyobs0 = noise_level * np.ones_like(xobs0)
from diffpy.srfit.fitbase import Profile
linedata = Profile()
linedata.setObservedProfile(xobs0, yobs0, dyobs0)

The next step is to create a fit contribution object, which will contain both the measured data and a
function calculator to simulate the data.

In [2]:
from diffpy.srfit.fitbase import FitContribution
linefit = FitContribution('linefit')
linefit.setProfile(linedata)
linefit.setEquation("A * x + B")

By defining the function calculator with the setEquation method, SrFit automatically recognizes A and B
as parameters that will be used to model the data. We now create a fit recipe to hold our fit contribution
and tell the recipe that A and B are the variables we want to refine.

In [3]:
from diffpy.srfit.fitbase import FitRecipe
recipe = FitRecipe()
recipe.addContribution(linefit);
recipe.addVar(recipe.linefit.A);
recipe.addVar(recipe.linefit.B);

Once defined, the fit recipe acts like a simple python function that takes the values of the variables as input 
and returns the fit residual as output. Thus, it can be plugged into a variety of available 
optimization programs. We'll use the least squares optimizer from the scipy.optimize module with some
reasonable starting values for A and B.

In [4]:
from scipy.optimize import leastsq
recipe.A = 1.0
recipe.B = 2.0
recipe.clearFitHooks()
leastsq(recipe.residual, recipe.values)

print("recipe.A = ", recipe.A.value)
print("recipe.B = ", recipe.B.value)

recipe.A =  2.8407684402216278
recipe.B =  4.905964647496833


The fit converged and gave us values of A and B that are somewhat close to our original starting values. Here's
where things get interesting. Suppose we have a second data set and model that also depend on A and B. We can
add another fit contribution to the recipe and run a co-refinement. Let's now generate more data--this time
we'll use a a quadratic form and, just for fun, we'll add another parameter, C.

In [5]:
C = 8.0
xobs1 = np.linspace(-3.0, 3.0, 31)
yobs1 = A * xobs1 ** 2 + B * xobs1 + C + noise_level * np.random.randn(xobs1.size)
dyobs1 = noise_level * np.ones_like(xobs0)

As before we create a fit contribution object to hold our measured data and the corresponding model.

In [6]:
quadfit = FitContribution('quadfit')
quaddata = Profile()
quaddata.setObservedProfile(xobs1, yobs1)
quadfit.setProfile(quaddata)
quadfit.setEquation("A * x**2 + B * x + C")

Now, creating a co-refinement is as simple as adding this second fit contribution to our existing recipe and
constraining the parameters A and B in our new equation to the existing variables A and B in our recipe.

In [7]:
recipe.addContribution(quadfit)
recipe.constrain(recipe.quadfit.A, recipe.A)
recipe.constrain(recipe.quadfit.B, recipe.B)
recipe.addVar(recipe.quadfit.C);

Once again we use scipy's least squares optimization engine to fit our data.

In [8]:
recipe.C = 1.0
leastsq(recipe.residual, recipe.values)
print("recipe.A = ", recipe.A.value)
print("recipe.B = ", recipe.B.value)
print("recipe.C = ", recipe.C.value)

recipe.A =  2.8529436829871115
recipe.B =  5.03015354415433
recipe.C =  7.859159101901894


With the added data we achieve a better agreement between our original parameters and the fit to the model.